In [1]:
import aiohttp
import asyncio
import pandas as pd
from datetime import datetime, timedelta

async def fetch_data_batch(session, url, headers, params, retries=3):
    """
    Fetches data for a batch of cryptocurrencies asynchronously with retries.
    """
    for attempt in range(retries):
        try:
            async with session.get(url, headers=headers, params=params) as response:
                if response.status == 200:
                    return await response.json()
                elif response.status == 429:
                    print("Rate limit hit. Waiting before retrying...")
                    await asyncio.sleep(10)  
                else:
                    print(f"Failed to fetch: HTTP {response.status}")
                    return None
        except Exception as e:
            print(f"Error fetching data: {str(e)}")
            await asyncio.sleep(5)
    return None

async def fetch_crypto_data_async(api_key, symbols):
    """
    Fetches cryptocurrency data for multiple symbols in batches asynchronously.
    """
    base_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
    headers = {
        'X-CMC_PRO_API_KEY': api_key,
        'Accept': 'application/json'
    }
    
    batch_size = 10
    symbol_batches = [symbols[i:i + batch_size] for i in range(0, len(symbols), batch_size)]
    tasks = []
    
    async with aiohttp.ClientSession() as session:
        for batch in symbol_batches:
            params = {
                'symbol': ','.join(batch),
                'convert': 'USD'
            }
            tasks.append(fetch_data_batch(session, base_url, headers, params))
        
        responses = await asyncio.gather(*tasks)
    
    all_data = []
    current_date = datetime.now().strftime('%Y-%m-%d')
    
    for response in responses:
        if response:
            try:
                for symbol, coin_data in response['data'].items():
                    quote_data = coin_data['quote']['USD']
                    record = {
                        'date': current_date,
                        'symbol': symbol,
                        'price': quote_data['price'],
                        'volume_24h': quote_data['volume_24h'],
                        'volume_change_24h': quote_data['volume_change_24h'],
                        'percent_change_1h': quote_data['percent_change_1h'],
                        'percent_change_24h': quote_data['percent_change_24h'],
                        'percent_change_7d': quote_data['percent_change_7d'],
                        'percent_change_30d': quote_data['percent_change_30d'],
                        'percent_change_60d': quote_data['percent_change_60d'],
                        'percent_change_90d': quote_data['percent_change_90d'],
                        'market_cap': quote_data['market_cap'],
                        'market_cap_dominance': quote_data['market_cap_dominance'],
                        'fully_diluted_market_cap': quote_data['fully_diluted_market_cap'],
                        'circulating_supply': coin_data.get('circulating_supply'),
                        'total_supply': coin_data.get('total_supply')
                    }
                    all_data.append(record)
            except KeyError as e:
                print(f"Error processing response: {e}")
    
    return pd.DataFrame(all_data)

async def create_historical_dataset_async(api_key, symbols, days=365):
    """
    Creates a dataset with historical data for the specified number of days asynchronously.
    """
    all_data = []
    current_date = datetime.now()
    
    for i in range(days):
        date = current_date - timedelta(days=i)
        print(f"Fetching data for {date.strftime('%Y-%m-%d')}...")
        
        df = await fetch_crypto_data_async(api_key, symbols)
        if not df.empty:
            df['date'] = date.strftime('%Y-%m-%d')
            all_data.append(df)
        
        await asyncio.sleep(2)
    
    final_df = pd.concat(all_data, ignore_index=True)
    final_df = final_df.sort_values(['price', 'date'])
    
    return final_df

async def main():
    API_KEY = "6377a87f-a8ee-4c6f-a62b-f63faefc3e20"
    symbols =  ["BTC", "ETH", "BNB", "USDT", "ADA", 
             "SOL", "AVAX", "MATIC", "LINK", "XRP", 
             "DOGEVERSE", "SEAL", "WAI", "SPONGEV2", "FLOKI"]
    historical_data = await create_historical_dataset_async(API_KEY, symbols, days=365)
    return historical_data

import nest_asyncio
nest_asyncio.apply()  

historical_data = asyncio.run(main())
historical_data.head()


Fetching data for 2025-01-02...
Fetching data for 2025-01-01...
Fetching data for 2024-12-31...
Fetching data for 2024-12-30...
Fetching data for 2024-12-29...
Fetching data for 2024-12-28...
Fetching data for 2024-12-27...
Fetching data for 2024-12-26...
Fetching data for 2024-12-25...
Fetching data for 2024-12-24...
Fetching data for 2024-12-23...
Fetching data for 2024-12-22...
Fetching data for 2024-12-21...
Fetching data for 2024-12-20...
Fetching data for 2024-12-19...
Fetching data for 2024-12-18...
Fetching data for 2024-12-17...
Fetching data for 2024-12-16...
Fetching data for 2024-12-15...
Fetching data for 2024-12-14...
Fetching data for 2024-12-13...
Fetching data for 2024-12-12...
Fetching data for 2024-12-11...
Fetching data for 2024-12-10...
Fetching data for 2024-12-09...
Fetching data for 2024-12-08...
Fetching data for 2024-12-07...
Fetching data for 2024-12-06...
Fetching data for 2024-12-05...
Fetching data for 2024-12-04...
Fetching data for 2024-12-03...
Fetching

,date,symbol,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,circulating_supply,total_supply
108,2024-12-26,DOGEVERSE,0.000011,5439.851909,104.061,2.217943,12.837254,16.310021,-31.290667,-48.793458,-49.063209,0.0,0.0,2224922.39,0.0,2.000000e+11
94,2024-12-27,DOGEVERSE,0.000011,5439.851909,104.061,2.217943,12.837254,16.310021,-31.290667,-48.793458,-49.063209,0.0,0.0,2224922.39,0.0,2.000000e+11
80,2024-12-28,DOGEVERSE,0.000011,5439.851909,104.061,2.217943,12.837254,16.310021,-31.290667,-48.793458,-49.063209,0.0,0.0,2224922.39,0.0,2.000000e+11
66,2024-12-29,DOGEVERSE,0.000011,5439.851909,104.061,2.217943,12.837254,16.310021,-31.290667,-48.793458,-49.063209,0.0,0.0,2224922.39,0.0,2.000000e+11
52,2024-12-30,DOGEVERSE,0.000011,5439.851909,104.061,2.217943,12.837254,16.310021,-31.290667,-48.793458,-49.063209,0.0,0.0,2224922.39,0.0,2.000000e+11


In [2]:
historical_data.to_csv('coindata.csv')

In [4]:
df = pd.read_csv('coindata.csv')
df.describe()

,Unnamed: 0,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,circulating_supply,total_supply
count,5110.000000,5110.000000,5.110000e+03,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5.110000e+03,5110.000000,5.110000e+03,5.110000e+03,5.110000e+03
mean,2554.500000,7253.026896,1.351442e+10,84.225428,0.517851,7.516885,9.894528,-1.745500,105.962861,475.959421,2.061116e+11,6.030397,2.260807e+11,7.024872e+11,7.264552e+11
std,1475.274268,24936.468000,2.790993e+10,82.974505,0.967497,4.441910,13.144233,38.250076,133.650979,1470.252005,4.882953e+11,14.285604,5.157677e+11,2.468554e+12,2.481916e+12
min,0.000000,0.000011,5.439782e+03,-18.938300,-1.245967,-0.048914,-2.742910,-35.313520,-48.793458,-49.063209,0.000000e+00,0.000000,2.224922e+06,0.000000e+00,1.980473e+07
25%,1277.250000,0.498831,6.897065e+06,43.211000,-0.003284,4.284448,1.803887,-20.099391,29.089398,27.480096,9.371048e+08,0.027400,1.790789e+09,1.980473e+07,1.204752e+08
50%,2554.500000,1.708536,1.280900e+09,65.175050,0.354825,7.852316,5.139098,-6.201385,57.326175,43.668348,2.491419e+10,0.730700,3.569912e+10,4.463683e+08,7.956692e+08
75%,3831.750000,207.436583,1.010157e+10,115.723300,0.839037,9.547808,11.943735,-0.124775,109.195739,101.002565,1.373202e+11,4.016600,1.413244e+11,3.513978e+10,9.998674e+10
max,5109.000000,97454.535636,1.054920e+11,340.916700,3.394859,16.869658,52.020350,129.382091,431.594698,5783.994741,1.930061e+12,56.504500,2.046545e+12,9.601110e+12,9.671506e+12


In [5]:
df["Volatility"] = df[["percent_change_1h", "percent_change_24h", "percent_change_7d"]].std(axis=1)

df["Normalized_Volatility"] = df["Volatility"] / df["Volatility"].max()
df["Normalized_MarketCapDominance"] = 1 / (df["market_cap_dominance"] + 1e-6) 
df["Normalized_VolumeChange"] = 1 - df["volume_change_24h"] / df["volume_change_24h"].max()

weights = {
    "Normalized_Volatility": 0.4,  
    "Normalized_MarketCapDominance": 0.4, 
    "Normalized_VolumeChange": 0.2, 
}

df["Composite_Score"] = (
    weights["Normalized_Volatility"] * df["Normalized_Volatility"] +
    weights["Normalized_MarketCapDominance"] * df["Normalized_MarketCapDominance"] +
    weights["Normalized_VolumeChange"] * df["Normalized_VolumeChange"]
)

risk_thresholds = {
    "Low": df["Composite_Score"].quantile(0.40),
    "Medium": df["Composite_Score"].quantile(0.60),
}

def classify_risk(score):
    if score <= risk_thresholds["Low"]:
        return "Low"
    elif score <= risk_thresholds["Medium"]:
        return "Medium"
    else:
        return "High"

df["Risk"] = df["Composite_Score"].apply(classify_risk) 